In [1]:
pip install opencv-python
pip install pupil-apriltags 
pip install pyyaml
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from platform import python_version
print(python_version())
# 3.7.16 works for pupil apriltag library

In [33]:
# Drone Coordinate System: +X towards front of drone, +Y towards right of drone, +Z towards ground
# Apriltag Coordinate System: +X towards right of tag, +Y towards top of tag, +Z further from tag
# Desired Coordinate System: origin at drone camera, coordinate rotation same as apriltag coordinate system but rotated about tag z axis to align yaw with drone
# Note that zero yaw occurs when xy of drone and apriltag coordinate systems align

import cv2
import os
from pupil_apriltags import Detector
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm

# Camera Calibration Data
fxcal = 938.61258241
fycal = 937.70826823
cxcal = 640/2
cycal = 480/2
tag_size = 107.5

cap = cv2.VideoCapture(0) # webcam reference
i = 0
x_vals = []
y_vals = []

# Check if opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while (cap.isOpened()):
    ret, frame = cap.read() # ret = bool for capture success, if success is stored in frame
    frame = cv2.resize(frame, None, fx=1, fy=1, interpolation=cv2.INTER_AREA)
    # frame is each image from webcam

    gray_img = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
   
    # initialize detector
    at_detector = Detector(
        families="tag36h11",
        nthreads=1,
        quad_decimate=1.0,
        quad_sigma=0.0,
        refine_edges=1,
        decode_sharpening=0.25,
        debug=0
    )

    tags = at_detector.detect(gray_img, estimate_tag_pose=True, camera_params=(fxcal, fycal, cxcal, cycal), tag_size=tag_size)
    for tag in tags:
        pose_r = tag.pose_R
        pose_t = -tag.pose_t
        homography = tag.homography
        pitch = np.arcsin(-pose_r[2,0]) # pitch dependent on tag orientation
        roll = np.arcsin(pose_r[2,1]/np.cos(pitch)) # roll depedent on tag orientation
        yaw = np.arcsin(pose_r[1,0]/np.cos(pitch))
        
#         Print apriltag position with drone as origin
#         print('x: ', np.round(pose_t[0],2), 'y: ', np.round(pose_t[1],2), 'z: ', np.round(pose_t[2],2), 'yaw: ', np.round(yaw,2), 'pitch: ', np.round(pitch,2),'roll: ', np.round(roll,2), end='\r')
        
    
#         Calculate and print the drone position with apriltag as origin
#         arr1 = np.array(pose_r)
#         cpose_r = np.linalg.inv(arr1)
#         cpose_t = np.array(-1*pose_t)
#         tframepose_t = np.matmul(cpose_r, cpose_t)
#         print('x: ', tframepose_t[0], 'y: ', tframepose_t[1], 'z: ', tframepose_t[2], end='\r')


#         Calculate the apriltag position with camera as origin, but with apriltag coordinate system rotated to align with drone yaw
#         arr1 = np.array(pose_r)
#         cpose_r = np.linalg.inv(arr1)
#         cpose_t = np.array(-1*pose_t)
#         tframepose_t = np.matmul(cpose_r, cpose_t)
#         cframepose_t = -tframepose_t
#         yawarray = np.array([[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]])
#         position = np.matmul(yawarray, cframepose_t)
#         print('x: ', position[0], 'y: ', position[1], 'z: ', position[2], 'yaw: ', -yaw, end='\r')
    
    tags = at_detector.detect(gray_img)
    for tag in tags:
#         print(int(tag.center[0]),int(tag.center[1]))
        detected_img = cv2.circle(gray_img, (int(tag.center[0]),int(tag.center[1])), radius=5, color=(200,200,200), thickness=2)
        for corner in tag.corners:
            detected_img = cv2.circle(gray_img, (int(corner[0]), int(corner[1])),radius=5, color=(0,255,0), thickness=2)    
        
#     print center for troubleshooting
    x_vals.append(int(corner[0]))
    y_vals.append(int(corner[1]))
    
#     for troubleshooting display on laptop    
    if type(detected_img) == np.ndarray:
        cv2.imshow("Filtered: Ecs to exit", detected_img)
        save_img = detected_img
    else:
        cv2.imshow("Filtered: Ecs to exit", frame)
        save_img = frame
    detected_img = frame; # reset img definition
    
    c = cv2.waitKey(1) # display popup window
    if c == 27: # until escape is pressed
        break
        
    # writes to folder on desktop, rename as required 
#     cv2.imwrite(os.path.join("C:\\Users\\katok\\OneDrive\\Desktop\\ME 495\\AprilTag",'Frame'+str(i)+'.jpg'), save_img) 
    i += 1
    
cap.release()
cv2.destroyAllWindows()